In [1]:
from pyspark.sql import SparkSession

### Option 1

In [2]:
spark= SparkSession.builder.appName("mybook").config("spark.master","local[*]").config("spark.driver.memory","4g").config("spark.sql.shuffle.partitions","5").config("spark.ui.port","4041").getOrCreate()

25/04/19 08:21:57 WARN Utils: Your hostname, Kartiks-Laptop.local resolves to a loopback address: 127.0.0.1; using 10.0.0.67 instead (on interface en0)
25/04/19 08:21:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/19 08:21:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
spark.stop()

### Option 2

In [60]:
spark_config={
    "spark.master":"local[4]",
    "spark.sql.shuffle.partitions":"4",
    "spark.driver.memory":"4g",
    "spark.ui.port":"4041"
}

In [ ]:
builder= SparkSession.builder.appName("mybook2")
for key, value in spark_config.items():
    builder=builder.config(key,value)

spark=builder.getOrCreate()

In [3]:
employeedata = spark.read.option("inferSchema","true").option("header","true").csv("/Users/kartikkumar/Downloads/acbb2271e66c10a5b73aacf82ca82784-e38afe62e088394d61ed30884dd50a6826eee0a8/employees.csv")

In [4]:
employeedata.take(3)

[Row(EMPLOYEE_ID=198, FIRST_NAME='Donald', LAST_NAME='OConnell', EMAIL='DOCONNEL', PHONE_NUMBER='650.507.9833', HIRE_DATE='21-JUN-07', JOB_ID='SH_CLERK', SALARY=2600, COMMISSION_PCT=' - ', MANAGER_ID='124', DEPARTMENT_ID=50),
 Row(EMPLOYEE_ID=199, FIRST_NAME='Douglas', LAST_NAME='Grant', EMAIL='DGRANT', PHONE_NUMBER='650.507.9844', HIRE_DATE='13-JAN-08', JOB_ID='SH_CLERK', SALARY=2600, COMMISSION_PCT=' - ', MANAGER_ID='124', DEPARTMENT_ID=50),
 Row(EMPLOYEE_ID=200, FIRST_NAME='Jennifer', LAST_NAME='Whalen', EMAIL='JWHALEN', PHONE_NUMBER='515.123.4444', HIRE_DATE='17-SEP-03', JOB_ID='AD_ASST', SALARY=4400, COMMISSION_PCT=' - ', MANAGER_ID='101', DEPARTMENT_ID=10)]

In [6]:
employeedata.sort("salary").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [salary#63 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(salary#63 ASC NULLS FIRST, 5), ENSURE_REQUIREMENTS, [plan_id=52]
      +- FileScan csv [EMPLOYEE_ID#56,FIRST_NAME#57,LAST_NAME#58,EMAIL#59,PHONE_NUMBER#60,HIRE_DATE#61,JOB_ID#62,SALARY#63,COMMISSION_PCT#64,MANAGER_ID#65,DEPARTMENT_ID#66] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/kartikkumar/Downloads/acbb2271e66c10a5b73aacf82ca82784-e38..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<EMPLOYEE_ID:int,FIRST_NAME:string,LAST_NAME:string,EMAIL:string,PHONE_NUMBER:string,HIRE_D...




In [4]:
from pyspark.sql.functions import col, column

In [8]:
employeedata.select(col("SALARY"))

DataFrame[SALARY: int]

In [9]:
employeedata.select("salary")

DataFrame[salary: int]

In [5]:
from pyspark.sql.functions import expr

In [11]:
employeedata.schema

StructType([StructField('EMPLOYEE_ID', IntegerType(), True), StructField('FIRST_NAME', StringType(), True), StructField('LAST_NAME', StringType(), True), StructField('EMAIL', StringType(), True), StructField('PHONE_NUMBER', StringType(), True), StructField('HIRE_DATE', StringType(), True), StructField('JOB_ID', StringType(), True), StructField('SALARY', IntegerType(), True), StructField('COMMISSION_PCT', StringType(), True), StructField('MANAGER_ID', StringType(), True), StructField('DEPARTMENT_ID', IntegerType(), True)])

In [12]:
expr("(salary+100)*2 < department_id")

Column<'(((salary + 100) * 2) < department_id)'>

In [13]:
employeedata.columns

['EMPLOYEE_ID',
 'FIRST_NAME',
 'LAST_NAME',
 'EMAIL',
 'PHONE_NUMBER',
 'HIRE_DATE',
 'JOB_ID',
 'SALARY',
 'COMMISSION_PCT',
 'MANAGER_ID',
 'DEPARTMENT_ID']

In [14]:
employeedata.first()

Row(EMPLOYEE_ID=198, FIRST_NAME='Donald', LAST_NAME='OConnell', EMAIL='DOCONNEL', PHONE_NUMBER='650.507.9833', HIRE_DATE='21-JUN-07', JOB_ID='SH_CLERK', SALARY=2600, COMMISSION_PCT=' - ', MANAGER_ID='124', DEPARTMENT_ID=50)

In [6]:
from pyspark.sql import Row

In [16]:
myrow=Row("Hello",None, 1,False)

In [17]:
myrow[2]

1

In [7]:
from pyspark.sql.types import StructType, StructField,StringType, LongType

In [19]:
myschema=StructType([
    StructField("id",StringType(),True),
    StructField("name", StringType(), True),
    StructField("salary", LongType(), True)
])

In [20]:
employeedata.select("employee_id","salary").show(2)

+-----------+------+
|employee_id|salary|
+-----------+------+
|        198|  2600|
|        199|  2600|
+-----------+------+
only showing top 2 rows



In [15]:
employeedata.select(
    expr("employee_id"),
    col("employee_id"),
    column("employee_id")
).show(2)

+-----------+-----------+-----------+
|employee_id|employee_id|employee_id|
+-----------+-----------+-----------+
|        198|        198|        198|
|        199|        199|        199|
+-----------+-----------+-----------+
only showing top 2 rows



In [12]:
employeedata.select(
    expr("employee_id as empid")
).show(2)

+-----+
|empid|
+-----+
|  198|
|  199|
+-----+
only showing top 2 rows



In [16]:
employeedata.select(
    expr("employee_id").alias("empid")
).show(2)

+-----+
|empid|
+-----+
|  198|
|  199|
+-----+
only showing top 2 rows



In [ ]:
spark

25/04/17 15:10:12 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 963330 ms exceeds timeout 120000 ms
25/04/17 15:10:12 WARN SparkContext: Killing executors is not supported by current scheduler.
25/04/17 15:15:22 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [18]:
employeedata.select("salary","employee_id").show(2)

+------+-----------+
|salary|employee_id|
+------+-----------+
|  2600|        198|
|  2600|        199|
+------+-----------+
only showing top 2 rows



In [19]:
employeedata.select(expr("salary AS sal")).show(2)

+----+
| sal|
+----+
|2600|
|2600|
+----+
only showing top 2 rows



In [20]:
employeedata.selectExpr("*").show(2)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
only showing top 2 rows



In [21]:
employeedata.selectExpr("*","(salary=employee_id) as withinsal").show(2)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+---------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|withinsal|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+---------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|    false|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|    false|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+---------+
only showing top 2 rows



In [22]:
employeedata.selectExpr("avg(salary)","count(distinct department_id)").show(2)

+-----------+-----------------------------+
|avg(salary)|count(DISTINCT department_id)|
+-----------+-----------------------------+
|    6182.32|                           10|
+-----------+-----------------------------+



In [8]:
from pyspark.sql.functions import lit

In [24]:
employeedata.select(expr("*"),lit(1).alias("one")).show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+---+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|one|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+---+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|  1|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|  1|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|  1|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|  1|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            -

In [25]:
employeedata.withColumn("numberone",lit(1)).show(2)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+---------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|numberone|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+---------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|        1|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|        1|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+---------+
only showing top 2 rows



In [26]:
employeedata.withColumn("withincountry", expr("1=2")).show(2)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|withincountry|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|        false|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|        false|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+-------------+
only showing top 2 rows



In [27]:
employeedata.withColumnRenamed("salary","sal").columns

['EMPLOYEE_ID',
 'FIRST_NAME',
 'LAST_NAME',
 'EMAIL',
 'PHONE_NUMBER',
 'HIRE_DATE',
 'JOB_ID',
 'sal',
 'COMMISSION_PCT',
 'MANAGER_ID',
 'DEPARTMENT_ID']

In [30]:
dfwithlongcolname=employeedata.withColumn("This Long Column-Name",expr('first_name'))

In [ ]:
dfwithlongcolname.selectExpr("`This Long Column-Name`","`This Long Column-Name` as `new col`").show(2)

+---------------------+-------+
|This Long Column-Name|new col|
+---------------------+-------+
|               Donald| Donald|
|              Douglas|Douglas|
+---------------------+-------+
only showing top 2 rows



25/04/18 15:47:30 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 918458 ms exceeds timeout 120000 ms
25/04/18 15:47:30 WARN SparkContext: Killing executors is not supported by current scheduler.
25/04/18 16:04:46 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [9]:
employeedata.select(expr("*"),lit(1).alias("one")).show(2)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+---+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|one|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+---+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|  1|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|  1|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+---+
only showing top 2 rows



In [10]:
employeedata.drop("commission_pct").columns

['EMPLOYEE_ID',
 'FIRST_NAME',
 'LAST_NAME',
 'EMAIL',
 'PHONE_NUMBER',
 'HIRE_DATE',
 'JOB_ID',
 'SALARY',
 'MANAGER_ID',
 'DEPARTMENT_ID']

In [13]:
employeedata.withColumn("sal",col("salary").cast("long"))

DataFrame[EMPLOYEE_ID: int, FIRST_NAME: string, LAST_NAME: string, EMAIL: string, PHONE_NUMBER: string, HIRE_DATE: string, JOB_ID: string, SALARY: int, COMMISSION_PCT: string, MANAGER_ID: string, DEPARTMENT_ID: int, sal: bigint]

In [14]:
employeedata.show(2)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
only showing top 2 rows



In [17]:
employeedata.filter(col("salary")<3000).show(2)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
only showing top 2 rows



In [18]:
employeedata.where("salary>2000").show(2)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
only showing top 2 rows



In [19]:
employeedata.where(col("salary")>2000).where(col("first_name")!="Donald").show(2)

+-----------+----------+---------+-------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|  EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+-------+------------+---------+--------+------+--------------+----------+-------------+
|        199|   Douglas|    Grant| DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen|JWHALEN|515.123.4444|17-SEP-03| AD_ASST|  4400|            - |       101|           10|
+-----------+----------+---------+-------+------------+---------+--------+------+--------------+----------+-------------+
only showing top 2 rows



In [21]:
employeedata.select("salary","department_id").distinct().count()

41

In [22]:
employeedata.sort("salary").show()

+-----------+----------+-----------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|  LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+-----------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        132|        TJ|      Olson| TJOLSON|650.124.8234|10-APR-07|ST_CLERK|  2100|            - |       121|           50|
|        128|    Steven|     Markle| SMARKLE|650.124.1434|08-MAR-08|ST_CLERK|  2200|            - |       120|           50|
|        136|     Hazel| Philtanker|HPHILTAN|650.127.1634|06-FEB-08|ST_CLERK|  2200|            - |       122|           50|
|        127|     James|     Landry| JLANDRY|650.124.1334|14-JAN-07|ST_CLERK|  2400|            - |       120|           50|
|        135|        Ki|        Gee|    KGEE|650.127.1734|12-DEC-07|ST_CLERK|  2400|            - |       122|           50|


In [23]:
employeedata.orderBy("salary").show(3)

+-----------+----------+----------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME| LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+----------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        132|        TJ|     Olson| TJOLSON|650.124.8234|10-APR-07|ST_CLERK|  2100|            - |       121|           50|
|        128|    Steven|    Markle| SMARKLE|650.124.1434|08-MAR-08|ST_CLERK|  2200|            - |       120|           50|
|        136|     Hazel|Philtanker|HPHILTAN|650.127.1634|06-FEB-08|ST_CLERK|  2200|            - |       122|           50|
+-----------+----------+----------+--------+------------+---------+--------+------+--------------+----------+-------------+
only showing top 3 rows



In [24]:
from pyspark.sql.functions import asc, desc

In [42]:
employeedata.orderBy(expr("salary").desc()).show()

+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID| FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        100|     Steven|     King|   SKING|515.123.4567|17-JUN-03|   AD_PRES| 24000|            - |        - |           90|
|        101|      Neena|  Kochhar|NKOCHHAR|515.123.4568|21-SEP-05|     AD_VP| 17000|            - |       100|           90|
|        102|        Lex|  De Haan| LDEHAAN|515.123.4569|13-JAN-01|     AD_VP| 17000|            - |       100|           90|
|        201|    Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        205|    Shelley|  Higgins|SHIGGINS|515.123.8080|07-JUN-02|    AC_MGR| 12008|            - |       101|       

In [36]:
employeedata.orderBy(col("salary").desc()).show()

+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID| FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        100|     Steven|     King|   SKING|515.123.4567|17-JUN-03|   AD_PRES| 24000|            - |        - |           90|
|        101|      Neena|  Kochhar|NKOCHHAR|515.123.4568|21-SEP-05|     AD_VP| 17000|            - |       100|           90|
|        102|        Lex|  De Haan| LDEHAAN|515.123.4569|13-JAN-01|     AD_VP| 17000|            - |       100|           90|
|        201|    Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        205|    Shelley|  Higgins|SHIGGINS|515.123.8080|07-JUN-02|    AC_MGR| 12008|            - |       101|       

In [44]:
employeedata.orderBy(col("salary").asc(),col("employee_id").desc()).show()

+-----------+----------+-----------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|  LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+-----------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        132|        TJ|      Olson| TJOLSON|650.124.8234|10-APR-07|ST_CLERK|  2100|            - |       121|           50|
|        136|     Hazel| Philtanker|HPHILTAN|650.127.1634|06-FEB-08|ST_CLERK|  2200|            - |       122|           50|
|        128|    Steven|     Markle| SMARKLE|650.124.1434|08-MAR-08|ST_CLERK|  2200|            - |       120|           50|
|        135|        Ki|        Gee|    KGEE|650.127.1734|12-DEC-07|ST_CLERK|  2400|            - |       122|           50|
|        127|     James|     Landry| JLANDRY|650.124.1334|14-JAN-07|ST_CLERK|  2400|            - |       120|           50|


In [46]:
employeedata.limit(4).show()

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03| AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|  MK_MAN| 13000|            - |       100|           20|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+



In [50]:
employeedata.orderBy(expr("salary desc")).limit(5).show(5)

+-----------+----------+----------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME| LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+----------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        132|        TJ|     Olson| TJOLSON|650.124.8234|10-APR-07|ST_CLERK|  2100|            - |       121|           50|
|        128|    Steven|    Markle| SMARKLE|650.124.1434|08-MAR-08|ST_CLERK|  2200|            - |       120|           50|
|        136|     Hazel|Philtanker|HPHILTAN|650.127.1634|06-FEB-08|ST_CLERK|  2200|            - |       122|           50|
|        127|     James|    Landry| JLANDRY|650.124.1334|14-JAN-07|ST_CLERK|  2400|            - |       120|           50|
|        135|        Ki|       Gee|    KGEE|650.127.1734|12-DEC-07|ST_CLERK|  2400|            - |       122|           50|
+-------

In [67]:
employeedata.rdd.getNumPartitions()

1

In [61]:
employeedata.repartition(5)

DataFrame[EMPLOYEE_ID: int, FIRST_NAME: string, LAST_NAME: string, EMAIL: string, PHONE_NUMBER: string, HIRE_DATE: string, JOB_ID: string, SALARY: int, COMMISSION_PCT: string, MANAGER_ID: string, DEPARTMENT_ID: int]

In [62]:
employeedata.repartition(col("salary"))

DataFrame[EMPLOYEE_ID: int, FIRST_NAME: string, LAST_NAME: string, EMAIL: string, PHONE_NUMBER: string, HIRE_DATE: string, JOB_ID: string, SALARY: int, COMMISSION_PCT: string, MANAGER_ID: string, DEPARTMENT_ID: int]

In [64]:
employeedata.repartition(5, col("salary"))

DataFrame[EMPLOYEE_ID: int, FIRST_NAME: string, LAST_NAME: string, EMAIL: string, PHONE_NUMBER: string, HIRE_DATE: string, JOB_ID: string, SALARY: int, COMMISSION_PCT: string, MANAGER_ID: string, DEPARTMENT_ID: int]

In [66]:
employeedata.repartition(5, col("salary")).coalesce(2)

DataFrame[EMPLOYEE_ID: int, FIRST_NAME: string, LAST_NAME: string, EMAIL: string, PHONE_NUMBER: string, HIRE_DATE: string, JOB_ID: string, SALARY: int, COMMISSION_PCT: string, MANAGER_ID: string, DEPARTMENT_ID: int]

In [70]:
#select 1, "one", 1.1
employeedata.select(lit(1),lit("one"),lit(1.1)).show()

+---+---+---+
|  1|one|1.1|
+---+---+---+
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
|  1|one|1.1|
+---+---+---+
only showing top 20 rows



In [77]:
employeedata.where(col("salary")!=2500).select("employee_id","first_name","salary").show(5, False)

+-----------+----------+------+
|employee_id|first_name|salary|
+-----------+----------+------+
|198        |Donald    |2600  |
|199        |Douglas   |2600  |
|200        |Jennifer  |4400  |
|201        |Michael   |13000 |
|202        |Pat       |6000  |
+-----------+----------+------+
only showing top 5 rows



In [81]:
employeedata.where(col("salary").isin("2600")).where("employee_id=199").show()

+-----------+----------+---------+------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME| EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+------+------------+---------+--------+------+--------------+----------+-------------+
|        199|   Douglas|    Grant|DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
+-----------+----------+---------+------+------------+---------+--------+------+--------------+----------+-------------+



In [82]:
from pyspark.sql.functions import instr

In [86]:
idfilter=col("employee_id")==199

In [84]:
employeedata.where(idfilter).show()

+-----------+----------+---------+------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME| EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+------+------------+---------+--------+------+--------------+----------+-------------+
|        199|   Douglas|    Grant|DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
+-----------+----------+---------+------+------------+---------+--------+------+--------------+----------+-------------+



In [ ]:
employeedata.withColumn("new_sal",col("salary")+100).where("new_sal>2600").select("employee_id","new_sal").show()

+-----------+-------+
|employee_id|new_sal|
+-----------+-------+
|        198|   2700|
|        199|   2700|
|        200|   4500|
|        201|  13100|
|        202|   6100|
|        203|   6600|
|        204|  10100|
|        205|  12108|
|        206|   8400|
|        100|  24100|
|        101|  17100|
|        102|  17100|
|        103|   9100|
|        104|   6100|
|        105|   4900|
|        106|   4900|
|        107|   4300|
|        108|  12108|
|        109|   9100|
|        110|   8300|
+-----------+-------+
only showing top 20 rows



25/04/19 20:06:14 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 960801 ms exceeds timeout 120000 ms
25/04/19 20:06:14 WARN SparkContext: Killing executors is not supported by current scheduler.
25/04/19 20:06:22 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$